In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
!pip install torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Operation cancelled by user


In [ ]:
import pandas as pd
from transformers import LongformerModel, AutoTokenizer
import torch as pt
from sklearn.model_selection import train_test_split
import os
import numpy as np
import chardet
from sklearn.metrics import accuracy_score, precision_score, recall_score,confusion_matrix
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


ImportError: ignored

In [ ]:
# Caricamento del tokenizer e del modello pre-addestrato CodeBERT
model = LongformerModel.from_pretrained("allenai/longformer-base-4096")
tokenizer = AutoTokenizer.from_pretrained("allenai/longformer-base-4096")

In [ ]:
dataframe = pd.read_csv("drive/MyDrive/Colab Notebooks/code smells tesi/Code_Smells_NLP-1/dataset/final_dataset.csv")


In [ ]:
dataframe

In [ ]:
import nltk
import re
from nltk.corpus import stopwords
nltk.download('stopwords')

stop_words = set(stopwords.words('italian'))

def remove_comments(code):
    # Rimuove i commenti su una singola riga
    code = re.sub(r'//.*', '', code)
    # Rimuove i commenti su più righe
    code = re.sub(r'/\*.*?\*/', '', code, flags=re.DOTALL)
    # Rimuove le stopwords
    tokens = [word.lower() for word in code.split() if word.lower() not in stop_words]
    filtered_text = ' '.join(tokens)
    # Rimuoviamo gli import
    code = re.sub(r'import\s+.*?;', '', code, flags=re.DOTALL)
    # Rimuoviamo i package
    code = re.sub(r'package\s+.*?;', '', code, flags=re.DOTALL)
    #Rimuove \n e \t
    code = re.sub(r'[\n\t]', '', code)
    return code

for i in range(len(dataframe)):
    dataframe['Component'][i] = remove_comments(dataframe['Component'][i])

In [ ]:
# Creazione X e y. X = dataset senza le etichette, y = etichette
dataframe2 = dataframe.drop(columns=['CDSBP','CC','LC','LZC','RB','SC'])
y = dataframe[['CDSBP','CC','LC','LZC','RB','SC']]

In [ ]:
# Divisione in train, validation e test set
X_train, X_val_test, Y_train, Y_val_test = train_test_split(dataframe2, y, test_size=0.4, random_state=1)
X_val, X_test, Y_val, Y_test = train_test_split(X_val_test, Y_val_test, test_size=0.5, random_state=1)

In [ ]:
# Tokenizzazione del set di addestramento
train_encodings = tokenizer(list(X_train['Component']), truncation=True, padding=True, max_length = 1024)
train_labels = pt.tensor(Y_train.values)

# Tokenizzazione del set di validazione
val_encodings = tokenizer(list(X_val['Component']), truncation=True, padding=True, max_length = 1024)
val_labels = pt.tensor(Y_val.values)

# Tokenizzazione del set di test
test_encodings = tokenizer(list(X_test['Component']), truncation=True, padding=True, max_length = 1024)
test_labels = pt.tensor(Y_test.values)

In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
import torch
from transformers import AdamW
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn

# Creazione del dataset di addestramento e validazione
train_dataset = TensorDataset(torch.tensor(train_encodings['input_ids']), train_labels)
val_dataset = TensorDataset(torch.tensor(val_encodings['input_ids']), val_labels)

# Creazione dei dataloader
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

# Creazione del modello
model.to(device)
model.train()

# Impostazione dell'ottimizzatore
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

# Definizione della funzione di perdita
loss_function = nn.CrossEntropyLoss()

# Addestramento del modello
for epoch in range(10):
    for batch in train_dataloader:
        optimizer.zero_grad()
        inputs = batch[0].to(device)
        labels = batch[1].to(device)

        outputs = model(inputs)
        logits = outputs.last_hidden_state[:, 0, :]

        # Modifica delle etichette in tensor 1D
        labels = torch.argmax(labels, dim=1)

        loss = loss_function(logits, labels)
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch+1} - Loss: {loss.item()}")

        # Liberazione memoria GPU
        del inputs, labels
        torch.cuda.empty_cache()

In [ ]:
torch.cuda.empty_cache()
from sklearn.metrics import accuracy_score, precision_score, jaccard_score, hamming_loss

val_accuracy = 0
total_val_samples = 0
precision_avg = 0
jaccard_score_avg = 0
hamming_loss_score = 0

for batch in val_dataloader:
    inputs = batch[0].to(device)
    labels = batch[1].to(device)

    val_outputs = model(inputs)
    val_predicted_labels = (val_outputs.pooler_output.sigmoid() > 0.5).int()  # Utilizza pooler_output invece di logits
    
    val_accuracy += (val_predicted_labels == labels).sum().item()
    precision_avg += precision_score(labels.cpu().numpy(), val_predicted_labels.cpu().numpy(), average='samples')
    jaccard_score_avg += jaccard_score(labels.cpu().numpy(), val_predicted_labels.cpu().numpy(), average='samples')
    hamming_loss_score += hamming_loss(labels.cpu().numpy(), val_predicted_labels.cpu().numpy())
    
    total_val_samples += len(inputs)
    # Libera la memoria GPU
    del inputs, labels, val_outputs, val_predicted_labels
    torch.cuda.empty_cache()

val_accuracy /= total_val_samples
precision_avg /= total_val_samples
jaccard_score_avg /= total_val_samples
hamming_loss_score /= total_val_samples

print(f"Validation Accuracy: {val_accuracy}")
print(f"Precision: {precision_avg}")
print(f"Jaccard Score: {jaccard_score_avg}")
print(f"Hamming Loss: {hamming_loss_score}")


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import multilabel_confusion_matrix

# Calcola le predizioni del modello sul dataset di validazione
model.eval()
with torch.no_grad():
    predicted_labels = []
    true_labels = []
    for batch in val_dataloader:
        inputs = batch[0].to(device)
        labels = batch[1].to(device)
        outputs = model(inputs)
        predicted_labels.extend(outputs.logits.sigmoid().round().cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Calcola la matrice di confusione
confusion_matrix = multilabel_confusion_matrix(true_labels, predicted_labels)

# Etichette delle classi
class_labels = ['CDSBP', 'CC', 'LC', 'LZC', 'RB', 'SC']

# Visualizza la matrice di confusione
plt.figure(figsize=(10, 8))
for i, cm in enumerate(confusion_matrix):
    plt.subplot(2, 3, i + 1)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=False)
    plt.xlabel('Etichetta predetta')
    plt.ylabel('Etichetta reale')
    plt.title(f'Matrice di confusione - {class_labels[i]}')
plt.tight_layout()
plt.show()
